In [50]:
import geopandas as gpd
import pandas as pd
import json
import numpy as np
import os
from shapely.geometry import Point
import math
from natsort import natsorted
from zipfile import ZipFile,ZIP_DEFLATED


In [51]:
STOCKHOLM_PROJ='EPSG:5850'
UNPROJECT='EPSG:4326'

In [52]:
grid_files = natsorted([x for x in os.listdir('../lib/actions/analysis/') if x.startswith('stock_nowater_pairs_400_')])

In [53]:
# grid_gdfs = [gpd.read_file('../lib/actions/analysis/' + x) for x in grid_files]
grid_gdfs = [json.load(open('../lib/actions/analysis/' + x))["pairs"] for x in grid_files]


In [42]:
# grid_centroids = [x['geometry'].unary_union.centroid for x in grid_gdfs]

In [55]:
# grid_xy = np.array([(pair.x,pair.y) for pair in grid_centroids])
grid_xy = [np.mean(points, axis=0) for points in grid_gdfs]
grid_bounds = [np.append(np.amax(points, axis=0), np.amin(points, axis=0)) for points in grid_gdfs]
print(grid_bounds[44])
print(grid_xy[44])

[18.21694819 59.32298716 18.11836833 59.25822916]
[18.16881176 59.28878743]


In [24]:
def toRad(value):
    return value * np.pi / 180;

def haversine(lat1,lon1,lat2,lon2):
    R = 6371
    latDistance = toRad(lat2-lat1)
    lonDistance = toRad(lon2-lon1)
    a = np.sin(latDistance / 2) * np.sin(latDistance / 2) + np.cos(toRad(lat1)) * np.cos(toRad(lat2)) * np.sin(lonDistance / 2) * np.sin(lonDistance / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c * 1000
    return distance


In [25]:
gtf_dir = 'gtfs/'

In [26]:
routes = pd.read_csv(gtf_dir+'routes.txt')
trips = pd.read_csv(gtf_dir+'trips.txt')
shapes = pd.read_csv(gtf_dir+'shapes.txt')
stops = pd.read_csv(gtf_dir+'stops.txt')
stop_times = pd.read_csv(gtf_dir+'stop_times.txt')
stops_re = stops.set_index('stop_id')

In [27]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,14010000543458876,23:55:00,23:55:00,9022001040004001,1,Slussen,3,1,0.00,1
1,14010000543458876,23:55:37,23:55:37,9022001040051002,2,Slussen,3,3,308.12,0
2,14010000543458876,23:56:27,23:56:27,9022001040049002,3,Slussen,3,3,557.90,0
3,14010000543458876,23:56:57,23:56:57,9022001040048001,4,Slussen,3,3,858.07,0
4,14010000543458876,23:57:28,23:57:28,9022001040034001,5,Slussen,3,3,1107.22,0


In [28]:
central_lat,central_lon = 59.331987,18.062266
central_pt = Point(central_lat,central_lon)

In [29]:
class stop_remover:
    
    def __init__(self,stops,stop_times,trips,lat,lon):
        self.stops = stops
        self.stop_times = stop_times
        self.trips = trips
        self.lat = lat
        self.lon = lon
        self.nearby_stops = None
        self.removed_stop_locs = None
        self.trip_ids_tp_remove = None
        
    def get_nearby_stops(self):
        nearby_stops = []
        for idx,(stoplat,stoplon) in self.stops.loc[:,['stop_lat','stop_lon']].iterrows():
            if haversine(self.lat,self.lon,stoplat,stoplon) < 2000:
                nearby_stops.append(self.stops.loc[idx,'stop_id'])
        self.nearby_stops = nearby_stops
        removed_stop_locs = self.stops.set_index('stop_id').loc[self.nearby_stops,['stop_lon','stop_lat']].values
        self.removed_stop_locs = removed_stop_locs
        return nearby_stops
    
    def removed_stops_to_file(self,filename):
        
        gpd.GeoSeries(gpd.points_from_xy(self.removed_stop_locs[:,0],self.removed_stop_locs[:,1])).to_file(filename,driver='GeoJSON')
    
    @staticmethod
    def get_centroid(arr):
        x_cent = np.mean(arr[:,0])
        y_cent = np.mean(arr[:,1])
        return np.array([x_cent,y_cent])

    
    def nearest_gridpt(self,gridlocs):
        arr_cent = self.get_centroid(self.removed_stop_locs)
        return np.argmin([np.linalg.norm(x-arr_cent) for x in gridlocs])
    
    
    def is_removed(self,x):
        return x in self.nearby_stops
    
    def clean_trips(self):
        self.trip_ids_to_remove = set(self.stop_times[self.stop_times['stop_id'].map(self.is_removed)]['trip_id'].values)
        
    
    



In [30]:
def get_edited_times(sr):
    def not_removed(x):
        return not sr.is_removed(x)
    return stop_times[stop_times['stop_id'].map(not_removed)]

In [31]:
def create_edited_times(lat,lon):
    sr = stop_remover(stops,stop_times,trips,lat,lon)
    sr.get_nearby_stops()
    stop_times_edited = get_edited_times(sr)
    stop_times_edited.to_csv('edited_gtfs/stop_times.txt',index=False)
    return sr

In [32]:
def write_gtfs():
    with ZipFile('edited_gtfs.zip', 'w',compression=ZIP_DEFLATED) as myzip:
        for file in os.listdir('edited_gtfs'):
            myzip.write(f'edited_gtfs/{file}')
        

In [33]:
kista_lat,kista_lon = 59.40109670140113, 17.94870101822394

In [34]:
sr = create_edited_times(kista_lat,kista_lon)
write_gtfs()
sr.nearest_gridpt(grid_xy) # what's the point of this? 

In [38]:
print(os.path.exists("./edited_gtfs/stop_times.txt"))

True


In [35]:
sr.removed_stops_to_file('removed_stops_17.json')

In [36]:
write_gtfs()

In [102]:
def is_nearby(stop):
    return 

def update_stop_times(nearby_stops):
    

IndentationError: expected an indented block (845420033.py, line 5)

In [ ]:
new_stop_times = stop_times

In [ ]:
trips_re = trips.set_index('trip_id')

In [ ]:
central_trips = trips_re.loc[all_trip_ids,:]

In [ ]:
central_trips

In [ ]:
central_routes = set(central_trips['route_id'].values)

In [ ]:
len(central_routes)

In [ ]:
len(nearby_stops)

In [ ]:
nearby_stops_str = [str(x) for x in nearby_stops]
central_routes_str = [str(x) for x in central_routes]

In [ ]:
sample_json = {
  "name": "testestes",
  "variants": [
    True,
    True,
    True
  ],
  "description": None,
  "feed": "602c4d1cfbd5464357cfafdc",
  "routes": central_routes_str,
  "trips": None,
  "stops": nearby_stops_str,
  "secondsSavedAtEachStop": -600
}

In [ ]:
with open('sample_stop_json.json','w+') as f:
    f.write(json.dumps(sample_json))

In [ ]:
stops_gdf = gpd.GeoDataFrame(stops,geometry=gpd.points_from_xy(stops.stop_lon,stops.stop_lat))

In [ ]:
stops_gdf.to_file('stops.json',driver='GeoJSON')

In [ ]:
stops.head()

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,platform_code
0,9021001070004000,Alhagsvägen,59.234952,17.851566,1,NaN,NaN
1,9021001070006000,Alby IP,59.237646,17.851894,1,NaN,NaN
2,9021001070010000,Albyvägen,59.237828,17.837123,1,NaN,NaN
3,9021001070012000,Hantverkarvägen,59.235563,17.836068,1,NaN,NaN
4,9021001070022000,Tegelbruksvägen,59.253833,17.868793,1,NaN,NaN


In [ ]:
shapes.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,1,59.251633,18.175625,1,0.00
1,1,59.251619,18.175669,2,0.00
2,1,59.251501,18.175583,3,14.00
3,1,59.251474,18.175509,4,19.21
4,1,59.251445,18.175397,5,26.31


In [ ]:
def group_shape(grouped_df):
    tot_dist = grouped_df['shape_dist_traveled']
shapes.groupby(by='shape_id')

In [ ]:
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_type
0,9011001019300000,14010000000001001,193,NaN,700
1,9011001055200000,14010000000001001,552,NaN,700
2,9011001011900000,14010000000001001,119,NaN,700
3,9011005098200000,14010000000002005,982,NaN,700
4,9011001065600000,14010000000001001,656,NaN,700


In [ ]:
set(routes['route_type'])

{100, 401, 700, 717, 900, 1000}

In [ ]:
len(routes)

664

### Route types
- 100: Railway Service
- 401: Metro Service
- 700: Bus Service
- 900: Tram Service
- 1000: Water Transport Service

In [ ]:
allowed_types = {700}

In [ ]:
allowed_routes = routes[routes['route_type'].map(lambda x: x in allowed_types)]

In [ ]:
len(allowed_routes)

577

In [ ]:
merge_trips = allowed_routes.merge(trips,how='left',left_on = 'route_id',right_on='route_id')

In [ ]:
merge_trips = merge_trips[~merge_trips['shape_id'].duplicated()]

In [ ]:
merge_shapes = merge_trips.merge(shapes,how='left',left_on='shape_id',right_on='shape_id')

In [ ]:
len(merge_shapes)

2022152

In [ ]:
gdf = gpd.GeoDataFrame(merge_shapes,geometry=gpd.points_from_xy(merge_shapes.shape_pt_lon,merge_shapes.shape_pt_lat))

In [ ]:
gdf.to_file('StockholmBusses.json',driver='GeoJSON')